# 基于T5的文本摘要

## Step1 导入相关包

In [ ]:
!pip install datasets transformers rouge-score nltk rouge_chinese

In [1]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-23 19:22:10.445675: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 19:22:10.484917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 19:22:11.109370: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./nlpcc_2017/")
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [3]:
ds = ds.train_test_split(200, seed=42)
ds

Loading cached split indices for dataset at /mnt/workspace/tra-code/02-NLP Tasks/15-text_summarization/nlpcc_2017/cache-f88b03791a18aede.arrow and /mnt/workspace/tra-code/02-NLP Tasks/15-text_summarization/nlpcc_2017/cache-6b0d5568bb085c36.arrow


DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 200
    })
})

In [4]:
ds["train"][0]

{'title': '郴州市发布雷电橙色预警:过去2小时北湖区、苏仙区、郴州市区、桂阳县、宜章县、嘉禾县、资兴市、桂东县、汝城县已经受...',
 'content': '发布日期:2015-03-3007:55:33郴州市气象台3月30日7时52分发布雷电橙色预警信号:过去2小时北湖区、苏仙区、郴州市区、桂阳县、宜章县、嘉禾县、资兴市、桂东县、汝城县已经受雷电活动影响,并将持续,出现雷电灾害事故的可能性比较大,请注意防范。图例标准防御指南2小时内发生雷电活动的可能性很大,或者已经受雷电活动影响,且可能持续,出现雷电灾害事故的可能性比较大。1、政府及相关部门按照职责落实防雷应急措施;2、人员应当留在室内,并关好门窗;3、户外人员应当躲入有防雷设施的建筑物或者汽车内;4、切断危险电源,不要在树下、电杆下、塔吊下避雨;5、在空旷场地不要打伞,不要把农具、羽毛球拍、高尔夫球杆等扛在肩上。'}

## Step3 数据处理

In [5]:
from modelscope import snapshot_download
model_dir = snapshot_download("langboat/mengzi-t5-base")

2024-07-23 19:22:19,001 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2024-07-23 19:22:19,003 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2024-07-23 19:22:19,004 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-23 19:22:19,026 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 e096d041fdba41d8d7d7331b3f469288 and a total number of 945 components indexed
2024-07-23 19:22:19,731 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


In [6]:
model_dir

'/mnt/workspace/.cache/modelscope/langboat/mengzi-t5-base'

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
def process_func(exmaples):
    contents = ["摘要生成: \n" + e for e in exmaples["content"]]
    inputs = tokenizer(contents, max_length=384, truncation=True)
    labels = tokenizer(text_target=exmaples["title"], max_length=64, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [9]:
tokenized_ds = ds.map(process_func, batched=True)
tokenized_ds

Loading cached processed dataset at /mnt/workspace/tra-code/02-NLP Tasks/15-text_summarization/nlpcc_2017/cache-1b523153b2e090f8.arrow
Loading cached processed dataset at /mnt/workspace/tra-code/02-NLP Tasks/15-text_summarization/nlpcc_2017/cache-a4ea025e3ee8adf8.arrow


DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [10]:
tokenizer.decode(tokenized_ds["train"][0]["input_ids"])

'摘要生成: 发布日期:2015-03-3007:55:33郴州市气象台3月30日7时52分发布雷电橙色预警信号:过去2小时北湖区、苏仙区、郴州市区、桂阳县、宜章县、嘉禾县、资兴市、桂东县、汝城县已经受雷电活动影响,并将持续,出现雷电灾害事故的可能性比较大,请注意防范。图例标准防御指南2小时内发生雷电活动的可能性很大,或者已经受雷电活动影响,且可能持续,出现雷电灾害事故的可能性比较大。1、政府及相关部门按照职责落实防雷应急措施;2、人员应当留在室内,并关好门窗;3、户外人员应当躲入有防雷设施的建筑物或者汽车内;4、切断危险电源,不要在树下、电杆下、塔吊下避雨;5、在空旷场地不要打伞,不要把农具、羽毛球拍、高尔夫球杆等扛在肩上。</s>'

In [11]:
tokenizer.decode(tokenized_ds["train"][0]["labels"])

'郴州市发布雷电橙色预警:过去2小时北湖区、苏仙区、郴州市区、桂阳县、宜章县、嘉禾县、资兴市、桂东县、汝城县已经受...</s>'

In [12]:
print(tokenized_ds["train"][0]["labels"])

[7, 27508, 5045, 1299, 1098, 580, 21296, 7701, 13, 888, 99, 1429, 512, 1080, 159, 6, 1193, 1707, 159, 6, 27508, 5045, 159, 6, 4449, 12817, 6, 2471, 761, 308, 6, 2351, 10099, 308, 6, 2476, 1345, 157, 6, 4449, 301, 308, 6, 7787, 17442, 147, 425, 1542, 1]


## Step4 创建模型

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

## Step5 创建评估函数

In [15]:
# !pip install rouge_chinese transformers==4.4.2

In [14]:
import numpy as np
from rouge_chinese import Rouge
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

rouge = Rouge()

def compute_metric(evalPred):
    def calculate_bleu_scores(candidate, references):
        candidate = list(candidate.split(" "))
        reference = [list(references.split(" "))]
        weights_list = [
            (1, 0, 0, 0),   # BLEU-1
            (0.5, 0.5, 0, 0),   # BLEU-2
            (1/3, 1/3, 1/3, 0),   # BLEU-3
            (0.25, 0.25, 0.25, 0.25) # BLEU-4
        ]

        bleu_scores = []
        for weights in weights_list:
            # print(sentence_bleu(reference, candidate, weights=weights))
            score = sentence_bleu(reference, candidate, weights=weights)
            bleu_scores.append(score)
        return bleu_scores
    predictions, labels = evalPred
    decode_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decode_preds = [" ".join(p) for p in decode_preds]
    decode_labels = [" ".join(l) for l in decode_labels]
    rouge_scores = rouge.get_scores(decode_preds, decode_labels, avg=True)
    bleu_scores_batch = np.mean(np.array([calculate_bleu_scores(cand, refs) for cand, refs in zip(decode_preds, decode_labels)]), axis=0)
    return {
        "rouge-1": rouge_scores["rouge-1"]["f"],
        "rouge-2": rouge_scores["rouge-2"]["f"],
        "rouge-l": rouge_scores["rouge-l"]["f"],
        "bleu-l": bleu_scores_batch[0],
        "bleu-2": bleu_scores_batch[1],
        "bleu-3": bleu_scores_batch[2],
        "bleu-4": bleu_scores_batch[3],
    }

In [17]:
rouge = Rouge()
decode_preds = [" ".join(p) for p in ["这是一份行动指南，确保军队始终服从党的命令", "确保军队始终服从党的命令"]]
decode_labels = [" ".join(l) for l in ["这是一份行动指南，确保军队永远听从党的指挥", "确保军队永远听从党的指挥"]]
print(decode_preds, decode_labels)
scores = rouge.get_scores(decode_preds, decode_labels, avg=True)
scores

['这 是 一 份 行 动 指 南 ， 确 保 军 队 始 终 服 从 党 的 命 令', '确 保 军 队 始 终 服 从 党 的 命 令'] ['这 是 一 份 行 动 指 南 ， 确 保 军 队 永 远 听 从 党 的 指 挥', '确 保 军 队 永 远 听 从 党 的 指 挥']


{'rouge-1': {'r': 0.6916666666666667,
  'p': 0.6726190476190477,
  'f': 0.6819105641071784},
 'rouge-2': {'r': 0.5772727272727273,
  'p': 0.5772727272727273,
  'f': 0.5772727222727273},
 'rouge-l': {'r': 0.6726190476190477,
  'p': 0.6726190476190477,
  'f': 0.6726190426190477}}

In [18]:
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
# 定义一个函数来计算所有四个BLEU分数



# 使用一个列表推导式来计算batch中每个候选句子的BLEU分数
bleu_scores_batch = np.mean(np.array([calculate_bleu_scores(cand, refs) for cand, refs in zip(decode_preds, decode_labels)]), axis=0)
bleu_dict = {f'bleu-{i+1}': score for i, score in enumerate(bleu_scores_batch)}
bleu_dict

NameError: name 'calculate_bleu_scores' is not defined

## Step6 配置训练参数

In [15]:
import matplotlib.pyplot as plt
from transformers import Seq2SeqTrainingArguments, TrainerCallback
from transformers.trainer_utils import TrainOutput

class MetricsLoggerCallback(TrainerCallback):
    def __init__(self):
        self.metrics = []
        self.losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero:
            loss = logs['loss']
            self.losses.append(loss)

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if state.is_local_process_zero:
            self.metrics.append(metrics)

# 创建MetricsLoggerCallback实例
metrics_logger_callback = MetricsLoggerCallback()

args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=10,
    logging_steps=6,
    evaluation_strategy='steps',
    save_strategy="epoch",
    metric_for_best_model="rouge-l",
    predict_with_generate=True,
    fp16=True
)

In [ ]:
help(Seq2SeqTrainingArguments)

## Step7 创建训练器

In [16]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)
)

## Step8 模型训练

In [17]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Bleu-l,Bleu-2,Bleu-3,Bleu-4
6,6.125700,3.840915,0.306185,0.146935,0.234592,0.188793,0.128804,0.081516,0.053708
12,3.890700,3.048959,0.335438,0.178075,0.269290,0.202171,0.147336,0.100510,0.070499
18,3.490000,2.841954,0.391786,0.221671,0.317150,0.242550,0.183743,0.133815,0.103092
24,3.195500,2.726521,0.410254,0.238629,0.333803,0.247245,0.190688,0.142789,0.110246
30,3.015000,2.653141,0.423811,0.257014,0.348157,0.266004,0.207881,0.160896,0.127823


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.8/site-packages/nltk/transla

In [ ]:
!tensorboard --logdir=./summary/runs/Jul23_19-22-44_dsw-583028-7856767748-clzf6 --export_scalars_to_csv=./summary/runs/scalars.csv

In [3]:
import os
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

def extract_scalars_from_event_file(event_file_path):
    ea = EventAccumulator(event_file_path)
    ea.Reload()

    wall_time = []
    step_nums = []
    values = []

    for tag in ea.Tags()['scalars']:
        for scalar_event in ea.Scalars(tag):
            wall_time.append(scalar_event.wall_time)
            step_nums.append(scalar_event.step)
            values.append(scalar_event.value)

    return {'wall_time': wall_time, 'step': step_nums, 'value': values}

def save_to_csv(data, file_name):
    import pandas as pd
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)

# 假设你有一个事件文件路径
event_file_path = '/summary/runs/Jul23_19-22-44_dsw-583028-7856767748-clzf6/events.out.tfevents.1721733777.dsw-583028-7856767748-clzf6.1957.0'

data = extract_scalars_from_event_file(event_file_path)
# save_to_csv(data, './summary/runs/scalars.csv')

NotFoundError: /summary/runs/Jul23_19-22-44_dsw-583028-7856767748-clzf6/events.out.tfevents.1721733777.dsw-583028-7856767748-clzf6.1957.0; No such file or directory

## Step9 模型推理

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe("摘要生成:\n" + ds["test"][-1]["content"], max_length=64, do_sample=True)

In [ ]:
ds["test"][-1]["title"]